参考资料：[Azure OpenAI 入门教程 - LangChain 篇 ：实战 - 构建企业内部知识库问答机器人 - 持久化向量存储](https://zhuanlan.zhihu.com/p/648167987)

In [12]:
! pip install psycopg2-binary
! pip install pgvector

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.2 MB 93.6 kB/s eta 0:00:12
   -- -----------

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\alexc\\AppData\\Roaming\\Python\\Python311\\site-packages\\psycopg2\\_psycopg.cp311-win_amd64.pyd'
Check the permissions.



Defaulting to user installation because normal site-packages is not writeable


In [1]:
import dotenv

dotenv.load_dotenv()

In [2]:
from langchain.document_loaders import DirectoryLoader
loader = DirectoryLoader("data", glob="**/*.txt")
documents = loader.load()

In [3]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
split_docs = text_splitter.split_documents(documents)

Created a chunk of size 107, which is longer than the specified 100
Created a chunk of size 144, which is longer than the specified 100
Created a chunk of size 104, which is longer than the specified 100


In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(deployment="lqembedding")

In [5]:
from langchain.vectorstores.chroma import Chroma

# 持久化数据
dosearch = Chroma.from_documents(documents=documents, embedding=embeddings, persist_directory="data/vector_store")
dosearch.persist()

In [6]:
# 加载数据
docsearch = Chroma(persist_directory="data/vector_store", embedding_function=embeddings)

Vector DB

In [10]:
import os
from langchain.vectorstores.pgvector import PGVector

connection_string = PGVector.connection_string_from_db_params(
    driver="psycopg2",
    host=os.getenv("DB_HOST"),
    port=int(os.getenv("DB_PORT")),
    database=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
)

import sqlalchemy
from sqlalchemy.orm import Session

# create vector extension
engine = sqlalchemy.create_engine(connection_string)
conn = engine.connect()
with Session(conn) as session:
    statement = sqlalchemy.text("CREATE EXTENSION IF NOT EXISTS vector")
    session.execute(statement)
    session.commit()

In [35]:
store = PGVector(collection_name="mycol", connection_string=connection_string, embedding_function=embeddings)

ids = [os.path.basename(doc.metadata["source"]) + ".part-" + str(index) for index, doc in enumerate(split_docs)]

store.add_documents(split_docs, ids=ids)

['long-text.txt.part.0',
 'long-text.txt.part.1',
 'long-text.txt.part.2',
 'long-text.txt.part.3',
 'long-text.txt.part.4',
 'long-text.txt.part.5',
 'long-text.txt.part.6',
 'long-text.txt.part.7',
 'long-text.txt.part.8',
 'long-text.txt.part.9',
 'long-text.txt.part.10',
 'long-text.txt.part.11',
 'long-text.txt.part.12',
 'long-text.txt.part.13',
 'long-text.txt.part.14',
 'long-text.txt.part.15',
 'long-text.txt.part.16']

In [19]:
docs = store.similarity_search("正高职称", include_metadata=True)

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

llm = ChatOpenAI(temperature=0)
chain = load_qa_chain(llm=llm, chain_type="refine", verbose=True)
result = chain.run(input_documents=docs, question="正高职称评定条件有哪些？")
print(result)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Context information is below.
------------
杭州工程师的职称评定条件（正高级工程师）

评审对象符合下列条件之一可申报正高级工程师职务任职资格：

1、取得高级工程师资格后，实际聘任高级工程师职务5年以上。
------------
Given the context information and not prior knowledge, answer any questions
Human: 正高职称评定条件有哪些？

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: 正高职称评定条件有哪些？
AI: 正高级工程师职称评定条件如下：

1. 取得高级工程师资格证书。
2. 实际聘任高级工程师职务满5年以上。
3. 在工程技术领域有较高的专业水平和技术能力。
4. 在工程项目中具有重要的技术贡献或者在科研、技术创新等方面有显著成果。
5. 在工程项目管理、技术指导、技术培训等方面有一定的经验和能力。
6. 具备良好的职业道德和职业操守，无违法违纪行为记录。
7. 具备较强的组织协调能力和团队合作精神。
8. 具备较强的学习能力和创新意识，能够不断提升自身的专业水平和技术能力。

以上是一般情况下的评定条件，具体要求可能会根据不同地区或单位的规定有所差异。
Human: We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
杭州工程师的职称评定条件（正高级工程师）

评审对象符合下列条件之一可申报正高级工程师职务任职资格：

1、取得高级工程师资格后，实际聘任高级工程师职务5年以上。
------------
Given the new context, refine the original answer t

In [21]:
print(result)

根据提供的新信息，杭州工程师的正高级工程师职称评定条件如下：

1. 取得高级工程师资格证书。
2. 实际聘任高级工程师职务满5年以上。
3. 实际聘任高级工程师职务后取得一项以上标志性业绩，或实际聘任工程师职务后取得二项以上标志性业绩。标志性业绩包括具有重要影响力、创新性、示范性的工程项目或科研成果。

以上是杭州工程师正高级工程师职称评定的详细条件。请注意，具体的评定标准可能会根据地区或单位的规定有所差异，建议咨询相关部门或机构以获取更准确的信息。
